# 통합 데이터 분석 (Comprehensive Data Analysis)

이 노트북은 다음 데이터셋들에 대한 포괄적인 분석을 제공합니다:

1. **농넷_시장별_사과가격.csv** - 도매시장 사과 가격 데이터
2. **초창패개발_데이터_판매데이터.csv** - 판매 거래 데이터
3. **초창패개발_데이터_판매량분석.csv** - 업체별 일자별 판매량 분석
4. **초창패개발_데이터_평균출고소요일.csv** - 업체별 평균 출고 소요 시간
5. **초창패개발_데이터_CS데이터.csv** - 고객 서비스(CS) 상세 데이터
6. **초창패개발_데이터_CS비율분석.csv** - 업체별 일자별 CS 비율

## 목차
1. 데이터 로드 및 전처리
2. 농넷 시장별 사과가격 분석
3. 판매 데이터 분석
4. 판매량 분석
5. 출고 소요일 분석
6. CS 데이터 분석
7. CS 비율 분석
8. 통합 인사이트 및 결론

In [ ]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 그래프 스타일 설정
sns.set_style('whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 6)

# Pandas 표시 옵션
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("라이브러리 로드 완료!")

---
# 1. 데이터 로드 및 전처리

In [ ]:
# 데이터 파일 경로 설정
data_dir = Path('../data/raw')

# 모든 CSV 파일 로드
print("=" * 80)
print("데이터 파일 로드 중...")
print("=" * 80)

# 1. 농넷 시장별 사과가격
df_apple_price = pd.read_csv(data_dir / '농넷_시장별_사과가격.csv', encoding='utf-8-sig')
print(f"✓ 농넷_시장별_사과가격.csv: {len(df_apple_price)} rows")

# 2. 판매 데이터
df_sales = pd.read_csv(data_dir / '초창패개발_데이터_판매데이터.csv', encoding='utf-8-sig')
print(f"✓ 초창패개발_데이터_판매데이터.csv: {len(df_sales)} rows")

# 3. 판매량 분석
df_sales_volume = pd.read_csv(data_dir / '초창패개발_데이터_판매량분석.csv', encoding='utf-8-sig')
print(f"✓ 초창패개발_데이터_판매량분석.csv: {len(df_sales_volume)} rows")

# 4. 평균 출고 소요일
df_delivery_days = pd.read_csv(data_dir / '초창패개발_데이터_평균출고소요일.csv', encoding='utf-8-sig')
print(f"✓ 초창패개발_데이터_평균출고소요일.csv: {len(df_delivery_days)} rows")

# 5. CS 데이터
df_cs = pd.read_csv(data_dir / '초창패개발_데이터_CS데이터.csv', encoding='utf-8-sig')
print(f"✓ 초창패개발_데이터_CS데이터.csv: {len(df_cs)} rows")

# 6. CS 비율 분석
df_cs_ratio = pd.read_csv(data_dir / '초창패개발_데이터_CS비율분석.csv', encoding='utf-8-sig')
print(f"✓ 초창패개발_데이터_CS비율분석.csv: {len(df_cs_ratio)} rows")

print("\n모든 데이터 로드 완료!")

---
# 2. 농넷 시장별 사과가격 분석

## 2.1 기본 정보 확인

In [ ]:
print("=" * 80)
print("농넷 시장별 사과가격 데이터 - 기본 정보")
print("=" * 80)
print(f"\n데이터 형태: {df_apple_price.shape}")
print(f"\n컬럼 목록:")
for col in df_apple_price.columns:
    print(f"  - {col}")

print("\n첫 10개 데이터:")
df_apple_price.head(10)

In [ ]:
# 데이터 타입 및 결측치 확인
print("=" * 80)
print("데이터 타입 및 결측치")
print("=" * 80)
df_apple_price.info()

## 2.2 데이터 전처리

In [ ]:
# 날짜 컬럼 변환
df_apple_price['DATE'] = pd.to_datetime(df_apple_price['DATE'])

# 숫자 컬럼에서 콤마 제거 및 숫자 변환
numeric_cols = ['평균가격', '총거래물량', '총거래금액']
for col in numeric_cols:
    if col in df_apple_price.columns:
        df_apple_price[col] = df_apple_price[col].astype(str).str.replace(',', '').astype(float)

print("데이터 전처리 완료!")
df_apple_price.head()

## 2.3 기본 통계량

In [ ]:
print("=" * 80)
print("수치형 변수 기본 통계량")
print("=" * 80)
df_apple_price[numeric_cols].describe()

## 2.4 결측치 분석

In [ ]:
# 결측치 분석
missing_apple = pd.DataFrame({
    '결측치 개수': df_apple_price.isnull().sum(),
    '결측치 비율(%)': (df_apple_price.isnull().sum() / len(df_apple_price) * 100).round(2)
})

print("=" * 80)
print("결측치 분석")
print("=" * 80)
print(missing_apple[missing_apple['결측치 개수'] > 0])

## 2.5 시장별, 품종별, 등급별 분석

In [ ]:
# 도매시장별 거래량
print("=" * 80)
print("도매시장별 거래 현황")
print("=" * 80)
market_stats = df_apple_price.groupby('도매시장').agg({
    '총거래물량': 'sum',
    '총거래금액': 'sum',
    '평균가격': 'mean'
}).round(0)
market_stats.columns = ['총 거래물량', '총 거래금액', '평균 가격']
print(market_stats.sort_values('총 거래물량', ascending=False))

In [ ]:
# 품종별 거래량
print("\n" + "=" * 80)
print("품종별 거래 현황")
print("=" * 80)
variety_stats = df_apple_price.groupby('품종').agg({
    '총거래물량': 'sum',
    '총거래금액': 'sum',
    '평균가격': 'mean'
}).round(0)
variety_stats.columns = ['총 거래물량', '총 거래금액', '평균 가격']
print(variety_stats.sort_values('총 거래물량', ascending=False).head(10))

In [ ]:
# 등급별 가격 분석
print("\n" + "=" * 80)
print("등급별 가격 현황")
print("=" * 80)
grade_stats = df_apple_price.groupby('등급').agg({
    '평균가격': ['mean', 'median', 'min', 'max'],
    '총거래물량': 'sum'
}).round(0)
print(grade_stats.sort_values(('평균가격', 'mean'), ascending=False))

## 2.6 가격 분포 시각화

In [ ]:
# 평균가격 분포 히스토그램
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 히스토그램
axes[0].hist(df_apple_price['평균가격'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(df_apple_price['평균가격'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Mean: {df_apple_price["평균가격"].mean():,.0f}')
axes[0].axvline(df_apple_price['평균가격'].median(), color='green', linestyle='--', 
                linewidth=2, label=f'Median: {df_apple_price["평균가격"].median():,.0f}')
axes[0].set_xlabel('Average Price (KRW)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Apple Prices')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 박스플롯 (등급별)
df_apple_price.boxplot(column='평균가격', by='등급', ax=axes[1])
axes[1].set_title('Price Distribution by Grade')
axes[1].set_xlabel('Grade')
axes[1].set_ylabel('Average Price (KRW)')
plt.suptitle('')

plt.tight_layout()
plt.show()

## 2.7 시계열 분석

In [ ]:
# 날짜별 평균 가격 추이
daily_price = df_apple_price.groupby('DATE').agg({
    '평균가격': 'mean',
    '총거래물량': 'sum'
}).reset_index()

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# 가격 추이
axes[0].plot(daily_price['DATE'], daily_price['평균가격'], marker='o', linewidth=2)
axes[0].set_ylabel('Average Price (KRW)')
axes[0].set_title('Daily Average Apple Price Trend')
axes[0].grid(True, alpha=0.3)

# 거래량 추이
axes[1].bar(daily_price['DATE'], daily_price['총거래물량'], alpha=0.7)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Total Volume')
axes[1].set_title('Daily Total Trading Volume')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2.8 산지별 분석

In [ ]:
# 산지별 거래 현황
if '산지-광역시도' in df_apple_price.columns:
    print("=" * 80)
    print("산지(광역시도)별 거래 현황")
    print("=" * 80)
    region_stats = df_apple_price.groupby('산지-광역시도').agg({
        '총거래물량': 'sum',
        '총거래금액': 'sum',
        '평균가격': 'mean'
    }).round(0)
    region_stats.columns = ['총 거래물량', '총 거래금액', '평균 가격']
    print(region_stats.sort_values('총 거래물량', ascending=False).head(10))
    
    # 시각화
    top_regions = region_stats.sort_values('총 거래물량', ascending=False).head(10)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 거래량
    axes[0].barh(top_regions.index, top_regions['총 거래물량'])
    axes[0].set_xlabel('Total Volume')
    axes[0].set_title('Top 10 Regions by Trading Volume')
    axes[0].grid(True, alpha=0.3)
    
    # 평균 가격
    axes[1].barh(top_regions.index, top_regions['평균 가격'], color='orange')
    axes[1].set_xlabel('Average Price (KRW)')
    axes[1].set_title('Top 10 Regions by Average Price')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

---
# 3. 판매 데이터 분석

## 3.1 기본 정보

In [ ]:
print("=" * 80)
print("판매 데이터 - 기본 정보")
print("=" * 80)
print(f"\n데이터 형태: {df_sales.shape}")
print(f"\n컬럼 목록:")
for col in df_sales.columns:
    print(f"  - {col}")

print("\n첫 10개 데이터:")
df_sales.head(10)

## 3.2 데이터 전처리

In [ ]:
# 발주날짜 변환
df_sales['발주날짜'] = pd.to_datetime(df_sales['발주날짜'])

# 공급가격에서 원화 기호 및 콤마 제거
df_sales['공급가격_숫자'] = df_sales['공급가격'].astype(str).str.replace('₩', '').str.replace(',', '').astype(float)

# 총 매출액 계산
df_sales['총매출액'] = df_sales['수량'] * df_sales['공급가격_숫자']

print("데이터 전처리 완료!")
df_sales.head()

## 3.3 기본 통계

In [ ]:
print("=" * 80)
print("판매 데이터 기본 통계")
print("=" * 80)
print(f"\n총 주문 건수: {len(df_sales):,}")
print(f"총 판매 수량: {df_sales['수량'].sum():,}")
print(f"총 매출액: ₩{df_sales['총매출액'].sum():,.0f}")
print(f"평균 주문 가격: ₩{df_sales['공급가격_숫자'].mean():,.0f}")
print(f"평균 주문 수량: {df_sales['수량'].mean():.2f}")

## 3.4 결측치 분석

In [ ]:
missing_sales = pd.DataFrame({
    '결측치 개수': df_sales.isnull().sum(),
    '결측치 비율(%)': (df_sales.isnull().sum() / len(df_sales) * 100).round(2)
})

print("=" * 80)
print("결측치 분석")
print("=" * 80)
print(missing_sales[missing_sales['결측치 개수'] > 0])

## 3.5 업체별 판매 현황

In [ ]:
# 업체별 통계
company_stats = df_sales.groupby('업체명').agg({
    '주문코드': 'count',
    '수량': 'sum',
    '총매출액': 'sum'
}).round(0)
company_stats.columns = ['주문 건수', '총 판매수량', '총 매출액']

print("=" * 80)
print("업체별 판매 현황")
print("=" * 80)
print(company_stats.sort_values('총 매출액', ascending=False))

In [ ]:
# 업체별 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 주문 건수
company_stats.sort_values('주문 건수', ascending=True)['주문 건수'].plot(kind='barh', ax=axes[0])
axes[0].set_xlabel('Order Count')
axes[0].set_title('Order Count by Company')
axes[0].grid(True, alpha=0.3)

# 판매 수량
company_stats.sort_values('총 판매수량', ascending=True)['총 판매수량'].plot(kind='barh', ax=axes[1], color='orange')
axes[1].set_xlabel('Total Sales Volume')
axes[1].set_title('Sales Volume by Company')
axes[1].grid(True, alpha=0.3)

# 매출액
company_stats.sort_values('총 매출액', ascending=True)['총 매출액'].plot(kind='barh', ax=axes[2], color='green')
axes[2].set_xlabel('Total Revenue (KRW)')
axes[2].set_title('Revenue by Company')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3.6 CS 여부 분석

In [ ]:
# CS 여부별 통계
cs_stats = df_sales.groupby('CS여부').agg({
    '주문코드': 'count',
    '수량': 'sum',
    '총매출액': 'sum'
}).round(0)
cs_stats.columns = ['주문 건수', '총 수량', '총 매출액']

print("=" * 80)
print("CS 여부별 통계")
print("=" * 80)
print(cs_stats)

if 'Y' in cs_stats.index and 'N' in cs_stats.index:
    cs_rate = (cs_stats.loc['Y', '주문 건수'] / cs_stats['주문 건수'].sum() * 100)
    print(f"\nCS 발생률: {cs_rate:.2f}%")

## 3.7 상품별 판매 분석

In [ ]:
# 상품별 판매 통계
product_stats = df_sales.groupby('판매상품명').agg({
    '주문코드': 'count',
    '수량': 'sum',
    '총매출액': 'sum'
}).round(0)
product_stats.columns = ['주문 건수', '총 판매수량', '총 매출액']

print("=" * 80)
print("상위 20개 인기 상품")
print("=" * 80)
print(product_stats.sort_values('주문 건수', ascending=False).head(20))

## 3.8 시계열 분석

In [ ]:
# 일자별 판매 추이
daily_sales = df_sales.groupby(df_sales['발주날짜'].dt.date).agg({
    '주문코드': 'count',
    '수량': 'sum',
    '총매출액': 'sum'
}).reset_index()
daily_sales.columns = ['날짜', '주문건수', '판매수량', '매출액']

fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# 주문 건수
axes[0].plot(daily_sales['날짜'], daily_sales['주문건수'], marker='o', linewidth=2)
axes[0].set_ylabel('Order Count')
axes[0].set_title('Daily Order Count Trend')
axes[0].grid(True, alpha=0.3)

# 판매 수량
axes[1].plot(daily_sales['날짜'], daily_sales['판매수량'], marker='o', linewidth=2, color='orange')
axes[1].set_ylabel('Sales Volume')
axes[1].set_title('Daily Sales Volume Trend')
axes[1].grid(True, alpha=0.3)

# 매출액
axes[2].bar(daily_sales['날짜'], daily_sales['매출액'], alpha=0.7, color='green')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Revenue (KRW)')
axes[2].set_title('Daily Revenue Trend')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
# 4. 판매량 분석

## 4.1 기본 정보

In [ ]:
print("=" * 80)
print("판매량 분석 데이터 - 기본 정보")
print("=" * 80)
print(f"\n데이터 형태: {df_sales_volume.shape}")
print("\n전체 데이터:")
df_sales_volume.head(20)

## 4.2 데이터 전처리

In [ ]:
# 첫 번째 컬럼을 날짜로 설정
date_col = df_sales_volume.columns[0]
df_sales_volume[date_col] = pd.to_datetime(df_sales_volume[date_col])
df_sales_volume = df_sales_volume.rename(columns={date_col: '날짜'})

print("데이터 전처리 완료!")
df_sales_volume.head()

## 4.3 업체별 총 판매량

In [ ]:
# 업체별 총 판매량
company_columns = [col for col in df_sales_volume.columns if col != '날짜']

total_sales_by_company = df_sales_volume[company_columns].sum().sort_values(ascending=False)

print("=" * 80)
print("업체별 총 판매량")
print("=" * 80)
print(total_sales_by_company)

# 시각화
plt.figure(figsize=(12, 6))
total_sales_by_company.plot(kind='bar', color='skyblue', edgecolor='black')
plt.xlabel('Company')
plt.ylabel('Total Sales Volume')
plt.title('Total Sales Volume by Company')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4.4 시계열 판매량 추이

In [ ]:
# 업체별 판매량 추이
plt.figure(figsize=(16, 8))

for company in company_columns:
    plt.plot(df_sales_volume['날짜'], df_sales_volume[company], 
             marker='o', label=company, linewidth=2)

plt.xlabel('Date')
plt.ylabel('Sales Volume')
plt.title('Sales Volume Trend by Company')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4.5 기본 통계량

In [ ]:
print("=" * 80)
print("업체별 판매량 통계")
print("=" * 80)
print(df_sales_volume[company_columns].describe())

---
# 5. 출고 소요일 분석

## 5.1 기본 정보

In [ ]:
print("=" * 80)
print("평균 출고 소요일 데이터")
print("=" * 80)
print(df_delivery_days)

## 5.2 시각화

In [ ]:
# 업체별 출고 소요일 비교
plt.figure(figsize=(12, 6))
plt.bar(df_delivery_days['업체명'], df_delivery_days['출고소요일'], 
        color=['green' if x == 1 else 'orange' for x in df_delivery_days['출고소요일']],
        edgecolor='black')
plt.xlabel('Company')
plt.ylabel('Delivery Days')
plt.title('Average Delivery Days by Company')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n출고 속도 분석:")
print(f"  - 1일 출고: {df_delivery_days[df_delivery_days['출고소요일'] == 1]['업체명'].tolist()}")
print(f"  - 2일 출고: {df_delivery_days[df_delivery_days['출고소요일'] == 2]['업체명'].tolist()}")

---
# 6. CS 데이터 분석

## 6.1 기본 정보

In [ ]:
print("=" * 80)
print("CS 데이터 - 기본 정보")
print("=" * 80)
print(f"\n데이터 형태: {df_cs.shape}")
print(f"총 CS 건수: {len(df_cs)}")
print("\n첫 10개 데이터:")
df_cs.head(10)

## 6.2 데이터 전처리

In [ ]:
# 발주일자 변환
df_cs['발주일자'] = pd.to_datetime(df_cs['발주일자'])

# 퍼센트 관련 컬럼 숫자로 변환
percent_cols = ['희망비율', '요청퍼센트', '셀러확정퍼센트', '업체확정퍼센트']
for col in percent_cols:
    if col in df_cs.columns:
        df_cs[col] = pd.to_numeric(df_cs[col], errors='coerce')

print("데이터 전처리 완료!")

## 6.3 업체별 CS 현황

In [ ]:
# 업체별 CS 건수
cs_by_company = df_cs['업체'].value_counts()

print("=" * 80)
print("업체별 CS 건수")
print("=" * 80)
print(cs_by_company)

# 시각화
plt.figure(figsize=(12, 6))
cs_by_company.plot(kind='bar', color='coral', edgecolor='black')
plt.xlabel('Company')
plt.ylabel('CS Count')
plt.title('CS Count by Company')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 6.4 셀러별 CS 현황

In [ ]:
# 셀러별 CS 건수
cs_by_seller = df_cs['셀러'].value_counts().head(10)

print("=" * 80)
print("상위 10개 셀러별 CS 건수")
print("=" * 80)
print(cs_by_seller)

## 6.5 CS 내용 분석

In [ ]:
# CS 내용 워드 분석 (간단한 키워드 빈도)
print("=" * 80)
print("CS 내용 샘플 (상위 10개)")
print("=" * 80)
for idx, content in enumerate(df_cs['CS내용'].head(10), 1):
    print(f"{idx}. {content[:100]}..." if len(str(content)) > 100 else f"{idx}. {content}")

## 6.6 환불 비율 분석

In [ ]:
# 요청 퍼센트 분포
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 히스토그램
axes[0].hist(df_cs['요청퍼센트'].dropna(), bins=20, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Request Percent (%)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Request Percent')
axes[0].grid(True, alpha=0.3)

# 확정 퍼센트 분포
axes[1].hist(df_cs['셀러확정퍼센트'].dropna(), bins=20, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_xlabel('Confirmed Percent (%)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Seller Confirmed Percent')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n환불 비율 통계:")
print(f"  평균 요청 퍼센트: {df_cs['요청퍼센트'].mean():.2f}%")
print(f"  평균 확정 퍼센트: {df_cs['셀러확정퍼센트'].mean():.2f}%")

## 6.7 시계열 CS 추이

In [ ]:
# 일자별 CS 발생 추이
cs_daily = df_cs.groupby(df_cs['발주일자'].dt.date).size().reset_index()
cs_daily.columns = ['날짜', 'CS 건수']

plt.figure(figsize=(16, 6))
plt.plot(cs_daily['날짜'], cs_daily['CS 건수'], marker='o', linewidth=2, color='red')
plt.xlabel('Date')
plt.ylabel('CS Count')
plt.title('Daily CS Occurrence Trend')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
# 7. CS 비율 분석

## 7.1 기본 정보

In [ ]:
print("=" * 80)
print("CS 비율 분석 데이터 - 기본 정보")
print("=" * 80)
print(f"\n데이터 형태: {df_cs_ratio.shape}")
print("\n전체 데이터:")
df_cs_ratio.head(20)

## 7.2 데이터 전처리

In [ ]:
# 첫 번째 컬럼을 날짜로 설정
date_col = df_cs_ratio.columns[0]
df_cs_ratio[date_col] = pd.to_datetime(df_cs_ratio[date_col])
df_cs_ratio = df_cs_ratio.rename(columns={date_col: '날짜'})

# 퍼센트 기호 제거 및 숫자 변환
cs_company_columns = [col for col in df_cs_ratio.columns if col != '날짜']
for col in cs_company_columns:
    df_cs_ratio[col] = df_cs_ratio[col].astype(str).str.replace('%', '').replace('', np.nan)
    df_cs_ratio[col] = pd.to_numeric(df_cs_ratio[col], errors='coerce')

print("데이터 전처리 완료!")
df_cs_ratio.head()

## 7.3 업체별 평균 CS 비율

In [ ]:
# 업체별 평균 CS 비율
avg_cs_ratio = df_cs_ratio[cs_company_columns].mean().sort_values(ascending=False)

print("=" * 80)
print("업체별 평균 CS 비율")
print("=" * 80)
for company, ratio in avg_cs_ratio.items():
    print(f"  {company}: {ratio:.4f}%")

# 시각화
plt.figure(figsize=(12, 6))
avg_cs_ratio.plot(kind='bar', color='tomato', edgecolor='black')
plt.xlabel('Company')
plt.ylabel('Average CS Ratio (%)')
plt.title('Average CS Ratio by Company')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 7.4 시계열 CS 비율 추이

In [ ]:
# 업체별 CS 비율 추이
plt.figure(figsize=(16, 8))

for company in cs_company_columns:
    plt.plot(df_cs_ratio['날짜'], df_cs_ratio[company], 
             marker='o', label=company, linewidth=2)

plt.xlabel('Date')
plt.ylabel('CS Ratio (%)')
plt.title('CS Ratio Trend by Company')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 7.5 CS 비율 통계

In [ ]:
print("=" * 80)
print("업체별 CS 비율 통계")
print("=" * 80)
print(df_cs_ratio[cs_company_columns].describe())

---
# 8. 통합 인사이트 및 결론

## 8.1 주요 발견사항 요약

In [ ]:
print("=" * 80)
print("데이터 분석 종합 요약")
print("=" * 80)

print("\n1. 농넷 시장별 사과가격")
print(f"   - 총 거래 레코드: {len(df_apple_price):,}")
print(f"   - 평균 가격: ₩{df_apple_price['평균가격'].mean():,.0f}")
print(f"   - 가격 범위: ₩{df_apple_price['평균가격'].min():,.0f} ~ ₩{df_apple_price['평균가격'].max():,.0f}")

print("\n2. 판매 데이터")
print(f"   - 총 주문 건수: {len(df_sales):,}")
print(f"   - 총 매출액: ₩{df_sales['총매출액'].sum():,.0f}")
print(f"   - 평균 주문 가격: ₩{df_sales['공급가격_숫자'].mean():,.0f}")

print("\n3. 판매량 분석")
print(f"   - 분석 기간: {df_sales_volume['날짜'].min().date()} ~ {df_sales_volume['날짜'].max().date()}")
print(f"   - 일평균 판매량: {df_sales_volume[company_columns].sum(axis=1).mean():.0f}")

print("\n4. 출고 소요일")
print(f"   - 평균 출고 소요일: {df_delivery_days['출고소요일'].mean():.1f}일")
print(f"   - 최단 출고: {df_delivery_days['출고소요일'].min()}일")
print(f"   - 최장 출고: {df_delivery_days['출고소요일'].max()}일")

print("\n5. CS 데이터")
print(f"   - 총 CS 건수: {len(df_cs)}")
print(f"   - 평균 환불 요청 비율: {df_cs['요청퍼센트'].mean():.2f}%")
print(f"   - 평균 확정 비율: {df_cs['셀러확정퍼센트'].mean():.2f}%")

print("\n6. CS 비율")
print(f"   - 전체 평균 CS 비율: {df_cs_ratio[cs_company_columns].mean().mean():.4f}%")
print(f"   - 최고 CS 비율 업체: {avg_cs_ratio.idxmax()} ({avg_cs_ratio.max():.4f}%)")
print(f"   - 최저 CS 비율 업체: {avg_cs_ratio.idxmin()} ({avg_cs_ratio.min():.4f}%)")

## 8.2 상관관계 분석 (판매량 vs CS 비율)

In [ ]:
# 날짜 기준으로 판매량과 CS 비율 병합
merged_data = pd.merge(
    df_sales_volume.set_index('날짜'),
    df_cs_ratio.set_index('날짜'),
    left_index=True,
    right_index=True,
    suffixes=('_sales', '_cs')
)

# 각 업체별 판매량과 CS 비율 상관관계
print("=" * 80)
print("업체별 판매량 vs CS 비율 상관계수")
print("=" * 80)

for company in company_columns:
    if f'{company}_sales' in merged_data.columns and f'{company}_cs' in merged_data.columns:
        corr = merged_data[f'{company}_sales'].corr(merged_data[f'{company}_cs'])
        print(f"  {company}: {corr:.4f}")

## 8.3 기계학습을 위한 권장사항

In [ ]:
print("=" * 80)
print("기계학습 모델링을 위한 권장사항")
print("=" * 80)

print("\n1. 데이터 전처리")
print("   - 날짜 데이터를 활용한 시계열 특성 생성 (요일, 주차, 월 등)")
print("   - 결측치 처리: 평균/중앙값 대체 또는 보간법 적용")
print("   - 범주형 변수 인코딩 (업체명, 상품명, 등급 등)")

print("\n2. 특성 엔지니어링")
print("   - 판매량 이동평균 (7일, 14일, 30일)")
print("   - 가격 변동률 계산")
print("   - 업체별 과거 CS 비율 누적 특성")
print("   - 출고 소요일을 고려한 배송 효율성 지표")

print("\n3. 예측 모델 제안")
print("   - 판매량 예측: 시계열 모델 (ARIMA, LSTM)")
print("   - CS 발생 예측: 분류 모델 (Random Forest, XGBoost)")
print("   - 가격 예측: 회귀 모델 (Linear Regression, Ridge, Lasso)")

print("\n4. 평가 지표")
print("   - 판매량 예측: MAE, RMSE, MAPE")
print("   - CS 발생 예측: Precision, Recall, F1-Score, AUC")
print("   - 가격 예측: R², MAE, RMSE")

print("\n5. 주의사항")
print("   - 데이터 불균형 문제 (CS 발생 건수가 적음) → SMOTE 등 적용 고려")
print("   - 계절성/추세 고려 (시계열 데이터)")
print("   - 외부 요인 (날씨, 공휴일 등) 추가 고려")

## 8.4 데이터 품질 체크리스트

In [ ]:
print("=" * 80)
print("데이터 품질 체크리스트")
print("=" * 80)

datasets = {
    '농넷_시장별_사과가격': df_apple_price,
    '판매데이터': df_sales,
    '판매량분석': df_sales_volume,
    '평균출고소요일': df_delivery_days,
    'CS데이터': df_cs,
    'CS비율분석': df_cs_ratio
}

for name, df in datasets.items():
    print(f"\n{name}:")
    print(f"  - 총 레코드 수: {len(df):,}")
    print(f"  - 컬럼 수: {len(df.columns)}")
    print(f"  - 결측치 비율: {(df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100):.2f}%")
    print(f"  - 중복 행: {df.duplicated().sum():,}")

print("\n" + "=" * 80)
print("분석 완료!")
print("=" * 80)